In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from sklearn.preprocessing import Normalizer, StandardScaler, RobustScaler,MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#import the data
pd.set_option('max_column', None)
pd.set_option('display.float_format', repr)
#pd.set_option('display.float_format', lambda x: '%.0f' % x)
original_data=pd.read_csv("D:/Stella/Documents/10_Academy/Week_1/Week-1/data/Week1_challenge_data.csv")
data=original_data.copy()
data.head()

In [ ]:
# column names in the dataset
data.columns.tolist()

In [ ]:
# number of data points
print(f" There are {data.shape[0]} rows and {data.shape[1]} columns")

# Check for missing values

In [ ]:
#total values missing in the dataset as a numerical value and as a percentage
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The Telco dataset contains", totalMissing," missing values or", round(((totalMissing/totalCells) * 100), 2), "%", "of the dataset.")

percent_missing(data)

In [ ]:
#Total percent of missing values per column
def missing_per_column(df):
    print(round((df.isna().sum()*100)/len(df), 2))
missing_per_column(data)

# Handling Missing Data
Missing values in the categorical columns will be replaced using the mode while the mean will be used for quantitative columns.
The histogram below shows some columns are skewed that may make the mean unsuitable for handling missing values.
However, the mean is used because the dataset will be normalized to make the values follow a normal distribution

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def exploratory_hist(df):
    df.hist(figsize=(15,15), layout=(11,5))
plt.show()
exploratory_hist(data)

In [ ]:
#plots = {i: px.histogram(data, x=i,) for i in data.columns}

In [ ]:
#plots['Bearer Id']

# Normalizing the data

In [ ]:
#impute missing values using the mean and mode
def impute(df):
    cat_list=['Bearer Id','Start','End','IMSI','MSISDN/Number','IMEI','Last Location Name','Handset Manufacturer','Handset Type']
    for i in df.columns:
        if i not in cat_list:
            df[i]=df[i].fillna(df[i].mean())
        else:
            df[i]=df[i].fillna(df[i].value_counts().index[0])
    return df

In [ ]:
impute(data).head()

In [ ]:
def normalizer(df):
    df_new=df.drop(['Bearer Id','Start','End','IMSI','MSISDN/Number','IMEI','Last Location Name','Handset Manufacturer','Handset Type'],axis=1)
    norm = Normalizer()
    #norm=RobustScaler()
    # normalize the data with boxcox
    normalized_data = norm.fit_transform(df_new)
    normalized_df=pd.DataFrame(normalized_data,columns = df_new.columns)
    cols_to_use = df.columns.difference(normalized_df.columns)
    final_df=pd.merge(df[cols_to_use],normalized_df, left_index=True, right_index=True, how='outer')
    #final_df=pd.concat([df,normalized_df],axis=1)
    return final_df
    #final_df.hist(figsize=(15,15), layout=(11,5))
    
normalizer(impute(data)).head()

In [ ]:
exploratory_hist(normalizer(impute(data)))

# Check data types

In [ ]:
#Convert start and end to date time
def date_time(df):
    df['Start']=pd.to_datetime(df['Start'], infer_datetime_format=True)
    df['End']=pd.to_datetime(df['End'], infer_datetime_format=True)
    return df
date_time(impute(data)).head()

In [ ]:
# check datatypes
(impute(data)).info()

# Cleaned data

In [ ]:
#check for missing values
missing_per_column(impute(data))

# Task 1.1

In [ ]:
clean_data=impute(data)
#clean_data=normalizer(impute(data))
clean_data.head()

In [ ]:
clean_data[clean_data["MSISDN/Number"] == 33664962239.0]['Bearer Id'].count()

In [ ]:
df_filtered= clean_data[(clean_data["MSISDN/Number"] == 33664962239)]
xDR_Count=df_filtered['Bearer Id'].nunique()
xDR_Count


In [ ]:
#Aggregate by xDR sessions per user
sessions_per_user=clean_data.groupby(["MSISDN/Number"],as_index=True)['Bearer Id'].count().reset_index().sort_values(['Bearer Id'],ascending=False).head(5)
sessions_per_user                     

In [ ]:
#Aggregate per user on session duration
duration_per_user=clean_data.groupby(["MSISDN/Number"],as_index=True)['Dur. (ms)'].sum().reset_index().sort_values(['Dur. (ms)'],ascending=False).head(5)
duration_per_user

In [ ]:
#Aggregate by Total DL and UL per user 
data_per_user=clean_data.groupby(["MSISDN/Number"],as_index=True).agg(Total_UL=("Total UL (Bytes)", sum),
                                        Total_DL=("Total DL (Bytes)", sum)).reset_index().sort_values(["Total_UL",
                                        "Total_DL"],ascending=False).head(5)
data_per_user

In [ ]:
#subset the data for applications
app_data=clean_data[['Bearer Id',"MSISDN/Number","Youtube DL (Bytes)","Youtube UL (Bytes)","Social Media DL (Bytes)","Social Media UL (Bytes)",
                   "Google DL (Bytes)","Google UL (Bytes)","Email DL (Bytes)","Email UL (Bytes)",
                   "Netflix DL (Bytes)","Netflix UL (Bytes)","Gaming DL (Bytes)","Gaming UL (Bytes)",
                   "Other DL (Bytes)","Other UL (Bytes)"]]

In [ ]:
app_data["Youtube"]=clean_data["Youtube DL (Bytes)"]+clean_data["Youtube UL (Bytes)"]
app_data["Social Media"]=clean_data["Social Media DL (Bytes)"]+clean_data["Social Media UL (Bytes)"] 
app_data["Google"]=clean_data["Google DL (Bytes)"]+clean_data["Google UL (Bytes)"]
app_data["Email"]=clean_data["Email DL (Bytes)"]+clean_data["Email UL (Bytes)"]
app_data["Netflix"]=clean_data["Netflix DL (Bytes)"]+clean_data["Netflix UL (Bytes)"]
app_data["Gaming"]=clean_data["Gaming DL (Bytes)"]+clean_data["Gaming UL (Bytes)"]
app_data["Other"]=clean_data["Other DL (Bytes)"]+clean_data["Other UL (Bytes)"]
#app_data.head()

In [ ]:
UL_DL_per_app=clean_data.groupby(["MSISDN/Number"]).agg(Youtube_DL=("Youtube DL (Bytes)",sum),
                                         Youtube_UL=("Youtube UL (Bytes)",sum),
                                         Social_Media_DL=("Social Media DL (Bytes)",sum),
                                         Social_Media_UL=("Social Media UL (Bytes)",sum),                           
                                         Google_DL=("Google DL (Bytes)",sum),
                                         Google_UL=("Google UL (Bytes)",sum),                          
                                         Email_DL=("Email DL (Bytes)",sum),
                                         Email_UL=("Email UL (Bytes)",sum),
                                         Netflix_DL=("Netflix DL (Bytes)",sum),
                                         Netflix_UL=("Netflix UL (Bytes)",sum),
                                         Gaming_DL=("Gaming DL (Bytes)",sum),
                                         Gaming_UL=("Gaming UL (Bytes)",sum),
                                         Other_DL=("Other DL (Bytes)",sum),
                                         Other_UL=("Other UL (Bytes)",sum)).head(6)
UL_DL_per_app

In [ ]:
#Aggregate by the total data volume (in Bytes) during this session for each application for each user
data_per_app=app_data.groupby(["MSISDN/Number"]).agg(Youtube=("Youtube",sum),
                                         Social_Media=("Social Media",sum),
                                         Google=("Google",sum),
                                         Email=("Email",sum),
                                         Netflix=("Netflix",sum),
                                         Gaming=("Gaming",sum),
                                         Other=("Other",sum)).head(6)
data_per_app

# Task 1.2

In [ ]:
#Obtaining the mean, median and standard deviation of the aggregated datasets
pd.DataFrame([["xDR_seesion_count",sessions_per_user["Bearer Id"].mean(),sessions_per_user["Bearer Id"].median(),
               sessions_per_user["Bearer Id"].std()],["Duration_per_user",duration_per_user["Dur. (ms)"].mean(),
            duration_per_user["Dur. (ms)"].median(),duration_per_user["Dur. (ms)"].std()],
              ["Upload_Data",data_per_user["Total_UL"].mean(),data_per_user["Total_UL"].median(),
               data_per_user["Total_UL"].std()],
             ["Download_Data",data_per_user["Total_DL"].mean(),data_per_user["Total_DL"].median(),
               data_per_user["Total_DL"].std()],
             ["Youtube Data",data_per_app["Youtube"].mean(),data_per_app["Youtube"].median(),data_per_app["Youtube"].std()],
             ["Social Media Data",data_per_app["Social_Media"].mean(),data_per_app["Social_Media"].median(),data_per_app["Social_Media"].std()],
             ["Google Data",data_per_app["Google"].mean(),data_per_app["Google"].median(),data_per_app["Google"].std()],
             ["Email Data",data_per_app["Email"].mean(),data_per_app["Email"].median(),data_per_app["Email"].std()],
             ["Netflix Data",data_per_app["Netflix"].mean(),data_per_app["Netflix"].median(),data_per_app["Netflix"].std()],
             ["Gaming Data",data_per_app["Gaming"].mean(),data_per_app["Gaming"].median(),data_per_app["Gaming"].std()],
             ["Other Data",data_per_app["Other"].mean(),data_per_app["Other"].median(),data_per_app["Other"].std()]],
              columns=['variable','mean','median','std'])

In [ ]:
# pd.DataFrame([["xDR_seesion_count",sessions_per_user["Bearer Id"].mean(),sessions_per_user["Bearer Id"].min(),
#                sessions_per_user["Bearer Id"].max(),sessions_per_user["Bearer Id"].median(),sessions_per_user["Bearer Id"].quantile(0.25),
#                sessions_per_user["Bearer Id"].quantile(0.75)],
#               ["Duration_per_user",duration_per_user["Dur. (ms)"].mean(),
#             duration_per_user["Dur. (ms)"].min(),duration_per_user["Dur. (ms)"].max(),duration_per_user["Dur. (ms)"].median(),
#                duration_per_user["Dur. (ms)"].quantile(0.25),duration_per_user["Dur. (ms)"].quantile(0.75),],
#               ["Upload_Data",data_per_user["Total_UL"].mean(),data_per_user["Total_UL"].min(),
#                data_per_user["Total_UL"].max(),data_per_user["Total_UL"].median(),data_per_user["Total_UL"].quantile(0.25),
#                data_per_user["Total_UL"].quantile(0.75)],
#              ["Download_Data",data_per_user["Total_DL"].mean(),data_per_user["Total_DL"].min(),
#                data_per_user["Total_DL"].max(),data_per_user["Total_DL"].median(),data_per_user["Total_DL"].quantile(0.25),data_per_user["Total_DL"].quantile(0.75)],
#              ["Youtube Data",data_per_app["Youtube"].mean(),data_per_app["Youtube"].min(),data_per_app["Youtube"].max(),data_per_app["Youtube"].median(),
#              data_per_app["Youtube"].quantile(0.25),data_per_app["Youtube"].quantile(0.75)],
#              ["Social Media Data",data_per_app["Social_Media"].mean(),data_per_app["Social_Media"].min(),data_per_app["Social_Media"].max(),
#              data_per_app["Social_Media"].median(),data_per_app["Social_Media"].quantile(0.25),data_per_app["Social_Media"].quantile(0.75)],
#              ["Google Data",data_per_app["Google"].mean(),data_per_app["Google"].min(),data_per_app["Google"].max(),
#              data_per_app["Google"].median(),data_per_app["Google"].quantile(0.25),data_per_app["Google"].quantile(0.75)],
#              ["Email Data",data_per_app["Email"].mean(),data_per_app["Email"].min(),data_per_app["Email"].max(),
#              data_per_app["Email"].median(),data_per_app["Email"].quantile(0.25),data_per_app["Email"].quantile(0.75)],
#              ["Netflix Data",data_per_app["Netflix"].mean(),data_per_app["Netflix"].median(),data_per_app["Netflix"].std(),
#              data_per_app["Netflix"].median(),data_per_app["Netflix"].quantile(0.25),data_per_app["Netflix"].quantile(0.75)],
#              ["Gaming Data",data_per_app["Gaming"].mean(),data_per_app["Gaming"].min(),data_per_app["Gaming"].max(),
#              data_per_app["Gaming"].median(),data_per_app["Gaming"].quantile(0.25),data_per_app["Gaming"].quantile(0.75)],
#              ["Other Data",data_per_app["Other"].mean(),data_per_app["Other"].min(),data_per_app["Other"].max(),
#              data_per_app["Other"].median(),data_per_app["Other"].quantile(0.25),data_per_app["Other"].quantile(0.75)]],
#               columns=['variable','mean','min','max','2Q','1Q','3Q'])

# Graphical Univariate Analysis

In [ ]:
app_filtered =  data_per_app[ data_per_app.index == 33601001754.0]

In [ ]:
np.array(app_filtered.iloc[0])

In [ ]:
#px.pie(app_filtered, values=np.array(app_filtered.iloc[0]), names=app_filtered.columns, title='Data Used Per Application')
px.pie(app_filtered, values=np.array(app_filtered.iloc[0]), names=app_filtered.columns, title='Data Used Per Application')


In [ ]:
#data_per_app.T.plot.pie(subplots=True, figsize=(11, 6))
#Pie chart showing data usage for six random users
wp = { 'linewidth' : 1, 'edgecolor' : "green" }

fig = plt.figure(figsize=(15,10))

for i, (name, row) in enumerate(data_per_app.iterrows()):
    ax = plt.subplot(3,2, i+1)
    ax.set_title(row.name)
    ax.set_aspect('equal')
    ax.pie(row, labels=row.index,
                                  shadow = True,autopct='%1.1f%%',
                                  wedgeprops = wp,
                                  textprops = dict(color ="black"))

plt.show()


In [ ]:
#pie chart showing which application uses the most data
explode = [0.8, 0.8, 0.5, 0.5, 0.8,0.2,0.2]
colors = ['tab:blue', 'tab:cyan', 'tab:gray', 'tab:orange', 'tab:red', 'tab:green','tab:brown']

data_per_app[data_per_app.columns[0:]].sum().plot.pie(shadow = True,autopct='%1.1f%%', colors = colors, explode = explode)

In [ ]:
#Bar chart comparing Total download and Total upload
data_per_user[data_per_user.columns[1:]].sum().plot.bar()

In [ ]:
#Boxplot to show position of outliers in count of sessions
sns.boxplot(sessions_per_user['Bearer Id'])

In [ ]:
#Histogram of duration spent in the sessions
plt.hist(duration_per_user['Dur. (ms)'])
plt.show()

In [ ]:
#Bivariate Analysis using stacked bar plot
axis = UL_DL_per_app[["Other_DL", "Other_UL"]].plot(kind="bar", stacked=True)
fig = axis.get_figure()

In [ ]:
#Variable Transformation

The correlation matrix indicates whether any relationship exists between the variables and the strength of that relationship.
In this case we would be looking at whether an increase in the data used in one application affects data used in another application. For example the relationship between data used in google versus data used in social media is a strong negative one. That means an increase in data used in google means less data will be used on social media.
The heatmap gives a good visual representation of the strength of the relationships, the lighter the colour the more positive the relationship and the darker the colour the more negative the relationship.

In [ ]:
#Correlation analysis
corr = data_per_app.corr()

In [ ]:
ax = sns.heatmap(corr, annot=True)

In [ ]:
#Dimensionality Reduction

# Task 2

In [ ]:
data2=original_data.copy()
clean_data2=impute(data2)
#clean_data2.head()

In [ ]:
#Aggregating by number of sessions
def count_sessions(df):
    sessions_per_user=pd.DataFrame(df.groupby(["MSISDN/Number"],as_index=True)['Bearer Id'].count()).sort_values(['Bearer Id'],ascending=False)
    return sessions_per_user 
count_sessions(clean_data2).head()

In [ ]:
def total_duration(df):
    duration_per_user=pd.DataFrame(df.groupby(["MSISDN/Number"],as_index=True)['Dur. (ms)'].sum()).sort_values(['Dur. (ms)'],ascending=False)
    return duration_per_user
total_duration(clean_data2).head(10)

In [ ]:
def total_data(df):
    df['Total Data Used']=df["Total DL (Bytes)"]+df["Total UL (Bytes)"]
    data_per_user=pd.DataFrame(df.groupby(["MSISDN/Number"],as_index=True)['Total Data Used'].sum()).sort_values(['Total Data Used'],ascending=False)
    return data_per_user
total_data(clean_data2).head()

# Normalize Each Metric
Use the Standard Scaler that ensures the data have a mean of 0 and variance of 1

In [ ]:
#create dataframe of all engagement metrics

engagement_df=reduce(lambda x,y: pd.merge(x,y, on='MSISDN/Number', how='outer'), [total_duration(clean_data2), total_data(clean_data2), count_sessions(clean_data2)])

In [ ]:
# def norm_df(df):
#     norm = Normalizer()
#     normalized_data = pd.DataFrame(norm.fit_transform(df))
#     return normalized_data 

# K-Means Clustering
K-Means is an unsupervised ML algorithm the finds clusters within the dataset. We begin with 3 initial clusters

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(engagement_df)

# statistics of scaled data
normalized_data=pd.DataFrame(data_scaled)

In [ ]:
# defining the kmeans function with initialization as k-means++
kmeans = KMeans(n_clusters=3, init='k-means++')

# fitting the k means algorithm on scaled data
kmeans.fit(data_scaled)

#predict the labels of clusters.
label = kmeans.fit_predict(data_scaled)

In [ ]:
# fitting multiple k-means algorithms and storing the values in an empty list
SSE = []
for cluster in range(1,20):
    kmeans = KMeans(n_jobs = -1, n_clusters = cluster, init='k-means++')
    kmeans.fit(data_scaled)
    SSE.append(kmeans.inertia_)

# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
#Getting unique labels
 
u_labels = np.unique(label)
 
#plotting the results:
 
for i in u_labels:
    plt.scatter(data_scaled[label == i , 0] , data_scaled[label == i , 1] , label = i)
plt.legend()
plt.show()

Cluster 0 and 2 are overlapping on the plot therefore we use a scree plot to determine the appropriate number of clusters.
Using the elbow method, the appropriate number of clusters is between two and five. Using two major two major clusters of people in the dataset.

In [ ]:
#K means clustering with two classes
# defining the kmeans function with initialization as k-means++
kmeans = KMeans(n_clusters=2, init='k-means++')

# fitting the k means algorithm on scaled data
kmeans.fit(data_scaled)

#predict the labels of clusters.
label = kmeans.fit_predict(data_scaled)
#Getting unique labels
 
u_labels = np.unique(label)
 
#plotting the results:
 
for i in u_labels:
    plt.scatter(data_scaled[label == i , 0] , data_scaled[label == i , 1] , label = i)
plt.legend()
plt.xlabel('total_duration')
plt.ylabel('total_data')
plt.show()

Cluster zero have low usage of both the first and second variable while cluster 1 have high usage of both variables in the dataset.(Referring to the first two variables in the dataset,duration and data used)

In [ ]:
#Add the clusters to the dataset
engagement_df['clusters'] = label.tolist()
engagement_df.head()

There are three variables in the dataset, a 3D plot would be needed to interpret the results, for ease of interpretation reduce the dimension of the data to two by PCA. According to PCA there are many users with low usage of both components.

In [ ]:
### Run PCA on the data and reduce the dimensions in pca_num_components dimensions

# Create a PCA model to reduce our data to 2 dimensions for visualization
pca = PCA(n_components=2)
pca.fit(data_scaled)

# Transform the scaled data to the new PCA space
X_reduced = pd.DataFrame(pca.transform(data_scaled),columns=['pca1','pca2'])
sns.scatterplot(x="pca1", y="pca2", data=X_reduced)
plt.title('K-means Clustering with 2 dimensions')
plt.show()

# Clusters metrics

In [ ]:
engagement_df.groupby('clusters').agg(['min', 'max', 'mean', 'sum'])

# User Data per Application

In [ ]:
def find_agg(df:pd.DataFrame, agg_column:str, agg_metric:str, col_name:str, top:int, order=False )->pd.DataFrame:
    
    new_df = df.groupby(agg_column)[col_name].agg(agg_metric).reset_index(name=col_name).\
                        sort_values(by=col_name, ascending=order)[:top]
    
    return new_df

In [ ]:
#Youtube top 10 users
find_agg(app_data,"MSISDN/Number","sum","Youtube",10)

In [ ]:
#Social_Media top 10 users
find_agg(app_data,"MSISDN/Number","sum","Social Media",10)

In [ ]:
#Google top 10 users
find_agg(app_data,"MSISDN/Number","sum","Google",10)

In [ ]:
#Email top 10 users
find_agg(app_data,"MSISDN/Number","sum","Email",10)

In [ ]:
#Netflix top 10 users
find_agg(app_data,"MSISDN/Number","sum","Netflix",10)

In [ ]:
#Gaming top 10 users
find_agg(app_data,"MSISDN/Number","sum","Gaming",10)

In [ ]:
#Other top 10 users
find_agg(app_data,"MSISDN/Number","sum","Other",10)

In [ ]:
#Plot the top 3 most used applications using appropriate charts
data_per_app.sum().nlargest(3).plot(kind='bar')
plt.title('Top 3 Apps')
plt.xlabel('App')
plt.ylabel('Data usage')
plt.show()

# TASK 3

In [ ]:
#outlier detection and replacement by mean
def outlier(df):
    column_name=['Avg RTT DL (ms)','Avg RTT UL (ms)','TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
                'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
                'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
                '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
                'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
                '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)']
    for i in column_name:
        upper_quartile=df[i].quantile(0.75)
        lower_quartile=df[i].quantile(0.25)
        df[i]=np.where(df[i]>upper_quartile,df[i].mean(),np.where(df[i]<lower_quartile,df[i].mean(),df[i]))
    return df

In [ ]:
clean_data3=outlier(clean_data)

In [ ]:
sns.boxplot(clean_data['DL TP < 50 Kbps (%)'],data=clean_data)

In [ ]:
#The two boxplots show that outliers are no longer present in the dataset.
sns.boxplot(clean_data3['DL TP < 50 Kbps (%)'],data=clean_data3)

In [ ]:
#Calculate total TCP,RTT and Throughput
clean_data3["Total TCP"]=clean_data3['TCP DL Retrans. Vol (Bytes)']+ clean_data3['TCP UL Retrans. Vol (Bytes)']+clean_data3['TCP DL Retrans. Vol (Bytes)']+ clean_data3['TCP UL Retrans. Vol (Bytes)']
clean_data3["Total RTT"]=clean_data3['Avg RTT DL (ms)']+clean_data3['Avg RTT UL (ms)']
clean_data3["Total Throughput"]=clean_data3['DL TP < 50 Kbps (%)']+clean_data3['50 Kbps < DL TP < 250 Kbps (%)']+clean_data3['250 Kbps < DL TP < 1 Mbps (%)']+clean_data3['DL TP > 1 Mbps (%)']+clean_data3['UL TP < 10 Kbps (%)']+clean_data3['10 Kbps < UL TP < 50 Kbps (%)']+clean_data3['50 Kbps < UL TP < 300 Kbps (%)']+clean_data3['UL TP > 300 Kbps (%)']

In [ ]:
# Average TCP per customer
#find_agg(clean_data3,"MSISDN/Number","mean","Total TCP",10)
avg_TCP=clean_data3.groupby('MSISDN/Number')['Total TCP'].mean().sort_values(ascending=False)


In [ ]:
#Average RTT per customer
#find_agg(clean_data3,"MSISDN/Number","mean","Total RTT",10)
avg_RTT=clean_data3.groupby('MSISDN/Number')['Total RTT'].mean().sort_values(ascending=False)

In [ ]:
#Average Throughput per customer
#find_agg(clean_data3,"MSISDN/Number","mean","Total Throughput",10)
avg_throughput=clean_data3.groupby('MSISDN/Number')['Total Throughput'].mean().sort_values(ascending=False)

In [ ]:
#Number of Handsets owned by an individual
#find_agg(clean_data3,"MSISDN/Number","count","Handset Type",10)
count_handsets=clean_data3.groupby(['MSISDN/Number','Handset Type'])['Handset Type'].count().sort_values(ascending=False)
count_handsets.head()

In [ ]:
#Top Ten Handsets used by manufacturer
#f = {Handset_Manufacturer_Count='Handset Manufacturer': 'count'}
clean_data3.groupby(['Handset Manufacturer','Handset Type']).agg(Handset_Manufacturer_Count=('Handset Manufacturer', 'count')).sort_values(['Handset Manufacturer','Handset_Manufacturer_Count'],ascending=False).head(10)

In [ ]:
#Top Ten Handsets used by customers
#f = {Handset_Manufacturer_Count='Handset Manufacturer': 'count'}
clean_data3.groupby(['MSISDN/Number','Handset Type']).agg(Handset_Type_Count=('Handset Type', 'count')).sort_values(['Handset_Manufacturer_Count'],ascending=False).head(10)

In [ ]:
#Top 3 Handset manufacturers
find_agg(clean_data3,"Handset Manufacturer","count","Handset Type",3)

In [ ]:
#top 5 handsets per top 3 handset manufacturer
options = ['Apple', 'Samsung', 'Huawei']
  
# selecting rows based on condition
top_3_df = clean_data3.loc[clean_data3['Handset Manufacturer'].isin(options)]
top_3_df.groupby(['Handset Manufacturer','Handset Type']).agg(Handset_Manufacturer_Count=('Handset Manufacturer', 'count')).sort_values(['Handset Manufacturer','Handset_Manufacturer_Count'],ascending=False).groupby('Handset Manufacturer').head(5)

In [ ]:
#most frequent TCP
clean_data3['Total TCP'].value_counts().head(1)

In [ ]:
#Top 10 RTT values
clean_data3['Total RTT'].value_counts().head(10)

In [ ]:
#Bottom 10 RTT values
clean_data3['Total RTT'].value_counts().tail(10)

In [ ]:
#Top 10 Throughput values
clean_data3['Total Throughput'].value_counts().head(10)
clean_data3['Total Throughput'].mean()

In [ ]:
#Bottom 10 Throughput values
clean_data3['Total Throughput'].value_counts().tail(10)

A histogram can be used to show distributionns of the experience metrics
Total throughput is left skewed so most handsets have an average throughput of 200
Most handsets also have a high average TCP

In [ ]:
clean_data3.groupby('Handset Manufacturer')['Total Throughput'].mean().plot(kind='hist')

In [ ]:
clean_data3.groupby('Handset Manufacturer')['Total TCP'].mean().plot(kind='hist')

# K-Means Clustering
The K-Means Cluster Plot is based on average RTT and average TCP. The people in cluster 0 tend to have a high RTT and a high TCP, the people in cluster 1 have a high RTT while the people in cluster 2 have a high TCP.

In [ ]:
experience_df=reduce(lambda x,y: pd.merge(x,y, on='MSISDN/Number', how='outer'), [avg_RTT,avg_TCP, avg_throughput, count_handsets])
experience_df.head()

In [ ]:
scaled_experience = scaler.fit_transform(experience_df)
# defining the kmeans function with initialization as k-means++
kmeans_exp = KMeans(n_clusters=3, init='k-means++')

# fitting the k means algorithm on scaled data
kmeans_exp.fit(scaled_experience)

#predict the labels of clusters.
label_exp = kmeans_exp.fit_predict(scaled_experience)

u_labels_exp = np.unique(label_exp)
 
#plotting the results:
 
for i in u_labels_exp:
    plt.scatter(scaled_experience[label_exp == i , 0] , scaled_experience[label_exp == i , 1] , label = i)
plt.legend()
plt.xlabel('Average RTT')
plt.ylabel('Average TCP')
plt.show()


# Task 4

In [ ]:
centroids = kmeans.cluster_centers_
engagement_score=euclidean_distances(data_scaled, centroids)
engagement_df['engagement_score']=[item[0] for item in engagement_score]
engagement_df.head()

In [ ]:
centroids[0]

In [ ]:
data_scaled

In [ ]:
centroids_exp = kmeans_exp.cluster_centers_
experience_score=euclidean_distances(scaled_experience, centroids_exp)
experience_df['experience_score']=[item[0] for item in experience_score]
experience_df.head()

In [ ]:
score_df=pd.merge(engagement_df[["engagement_score"]], experience_df[["experience_score"]], on="MSISDN/Number", how='outer')
score_df=score_df[~score_df.index.duplicated(keep='first')]
score_df['satisfaction_score'] = score_df.mean(axis=1)
score_df=score_df.sort_values(['satisfaction_score'],ascending=False)

# Regression and Prediction using Random Forest

In [ ]:
reg_df=reduce(lambda x,y: pd.merge(x,y, on='MSISDN/Number', how='outer'), [engagement_df, experience_df, score_df[['satisfaction_score']]]).drop(["engagement_score","experience_score","clusters"],axis=1)
reg_df=reg_df[~reg_df.index.duplicated(keep='first')]
reg_df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reg_df.drop(['satisfaction_score'], axis=1), reg_df['satisfaction_score'], test_size = 0.20, random_state=440)

In [ ]:
# creating scaler scale var.
norm = MinMaxScaler()
# fit the scale
norm_fit = norm.fit(X_train)
scal_xtrain = norm_fit.transform(X_train)

# transformation of testing data
scal_xtest = norm_fit.transform(X_test)
# create model variable
rnd = RandomForestRegressor(n_estimators = 100, random_state = 100)
  
# fit the model
fit_rnd = rnd.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import classification_report
x_predict = list(rnd.predict(X_test))
# Calculate the absolute errors
errors = abs(x_predict - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': x_predict}).sort_values(['Actual'],ascending=False)
df

# K-Means Clusters

In [ ]:
score_experience = scaler.fit_transform(score_df)
# defining the kmeans function with initialization as k-means++
kmeans_score = KMeans(n_clusters=2, init='k-means++')

# fitting the k means algorithm on scaled data
kmeans_score.fit(score_experience)

#predict the labels of clusters.
label_score = kmeans_score.fit_predict(score_experience)

u_labels_score = np.unique(label_score)
 
#plotting the results:
 
for i in u_labels_score:
    plt.scatter(score_experience[label_score == i , 0] , score_experience[label_score == i , 1] , label = i)
plt.legend()
plt.xlabel('Engagement Score')
plt.ylabel('Experience Score')
plt.show()

Aggregate the average satisfaction & experience score per cluster. 

In [ ]:
score_df['clusters'] = label_score.tolist()
score_df.groupby('clusters').agg(['mean'])